In [1]:
from tab_model import TabNetRegressor, TabModel
import torch
import numpy as np
import time
from tqdm import tqdm

/opt/miniconda3/envs/opence-v1.5.1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def getting_datapaths(
    train_idx,
    val_idx,
    files='sunny', 
    target_cols=["ccn_001", "ccn_003", "ccn_006"],
    exclude_cols = ["XLONG", "XLAT"],
):
    # getting colname
    featloc = "/home/kphadke/hackathon/norm_data_timesteps_feat"
    targloc = "/home/kphadke/hackathon/norm_data_timesteps_targ"

    featcols = np.load(f"{featloc}/x_name.npy")
    targcols = np.load(f"{targloc}/y_name.npy")
    
    target_idx = [np.where(targcols == i)[0].item() for i in target_cols ]
    feat_idx   = [np.where(featcols == i)[0].item() for i in featcols if i not in exclude_cols]
    
    # getting paths
    if files == 'sunny':
        # my data
        featloc = "/home/kwoksun2/hackathon_data/norm_data"
        targloc = "/home/kwoksun2/hackathon_data/norm_data"

        train_feat_files = [f"{featloc}/t{t}_feat.npy" for t in train_idx]
        val_feat_files   = [f"{featloc}/t{t}_feat.npy" for t in val_idx  ]

        train_targ_files = [f"{targloc}/t{t}_targ.npy" for t in train_idx]
        val_targ_files   = [f"{targloc}/t{t}_targ.npy" for t in val_idx]
    
    elif files == 'kedar':
        featloc = "/home/kphadke/hackathon/norm_data_timesteps_feat"
        targloc = "/home/kphadke/hackathon/norm_data_timesteps_targ"

        train_feat_files = [f"{featloc}/{t}feat_norm.npy" for t in train_idx]
        val_feat_files   = [f"{featloc}/{t}feat_norm.npy" for t in val_idx  ]

        train_targ_files = [f"{targloc}/{t}targ_norm.npy" for t in train_idx]
        val_targ_files   = [f"{targloc}/{t}targ_norm.npy" for t in val_idx]

    return [np.array(train_feat_files),np.array(train_targ_files)], [np.array(val_feat_files),np.array(val_targ_files)], feat_idx, target_idx

In [26]:
!ls ep_19*

ep_19_kedar_ccn_001,ccn_003,ccn_006_XLONG,XLAT_tabnet.pt
ep_19_kedar_D_ALPHA,D_GAMMA,D_ALPHA_CCN,D_GAMMA_CCN_XLONG,XLAT_tabnet.pt
ep_19_sunny_CHI,CHI_CCN_XLONG,XLAT_tabnet.pt


In [27]:
# ckpt_name = "ep_17_kedar_ccn_001,ccn_003,ccn_006_XLONG,XLAT_tabnet.pt"

ckpt_name = "ep_19_kedar_D_ALPHA,D_GAMMA,D_ALPHA_CCN,D_GAMMA_CCN_XLONG,XLAT_tabnet.pt"


import os

folder_name = f'figs_{ckpt_name.replace(".pt","")}'

if not os.path.exists(folder_name):
    os.mkdir(folder_name)
    
ckpt = torch.load(ckpt_name)

In [28]:
model = ckpt['model_state_dict']
datasource = ckpt['datasource']
exclude_cols =  ckpt['exclude_cols']
target_cols =  ckpt['target_cols']

In [29]:
# getting the idx used in the model
featloc = "/home/kphadke/hackathon/norm_data_timesteps_feat"
targloc = "/home/kphadke/hackathon/norm_data_timesteps_targ"

featcols = np.load(f"{featloc}/x_name.npy")
targcols = np.load(f"{targloc}/y_name.npy")

target_idx = [np.where(targcols == i)[0].item() for i in target_cols ]
feat_idx   = [np.where(featcols == i)[0].item() for i in featcols if i not in exclude_cols]

In [30]:
from sklearn.model_selection import train_test_split

# THIS IS JUST FOR TESTING!!!!!!!
# WILL CHANGE BACK TO 133 later
nsnapshots = 133

# DONT CHANGE THE SEEEDDDDDD!!!!!
np.random.seed(52)
full_indx = np.arange(nsnapshots)
train_idx, val_idx = train_test_split(full_indx, test_size=0.2, train_size=0.8) 

In [31]:
train_files, val_files, feat_idx, target_idx = getting_datapaths(
    train_idx, 
    val_idx, 
    files=datasource, 
    exclude_cols = exclude_cols,
    target_cols  = target_cols
)

In [32]:
# GETTING TESTSET DATA!!!!
# get the dataset in place, 
# make sure it has the appropriate normalizations (reading from the )
# 

In [33]:
# kedar
if datasource == 'kedar':
    test_feats = [f"/home/kphadke/hackathon/norm_data_timesteps_feat_eval/{i}feat_norm_eval.npy" for i in range(12)]
    test_targs = [f"/home/kphadke/hackathon/norm_data_timesteps_targ_eval/{i}targ_norm_eval.npy" for i in range(12)]
else:
    test_feats = [f"/home/kwoksun2/hackathon_data/norm_data_val/t{i}_feat.npy" for i in range(12)]
    test_targs = [f"/home/kwoksun2/hackathon_data/norm_data_val/t{i}_targ.npy" for i in range(12)]
#     raise ValueError
    
if datasource == "sunny":
    target_std= np.array(
        [1.3428156,
        1.0779657,
        1.0479535,
        0.08097392,
        0.10943509 ,
        0.20763765 ,
        0.2632665 ,
        0.07147439,
        0.091097355,
        1.1029556
        ])
    
    tstd = target_std[target_idx]
    
elif datasource == "kedar":
    target_std = np.load("/home/kphadke/hackathon/targ_std.npy").transpose()
    tstd = target_std[:,target_idx]
    tstd = tstd[:,np.newaxis,np.newaxis,:]

In [34]:
test_files = [test_feats, test_targs]

In [35]:
import os

folder_name = f'predictions_{ckpt_name.replace(".pt","")}'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

In [36]:
# create a prediction dump
feat_files, targ_files = test_files
# we are selecting only the first validation data
for i in range(len(feat_files)):
    feat_data = np.load(feat_files[i])
    targ_data = np.load(targ_files[i])

    if targ_data.ndim == 1:
        targ_data = targ_data.reshape(10,-1).transpose()

    tds = torch.utils.data.TensorDataset(
        torch.from_numpy(feat_data[:,feat_idx]).float(), 
        torch.from_numpy(targ_data[:,target_idx]).float()
    )
    dl = torch.utils.data.DataLoader(tds, batch_size=1024, shuffle=False)

    # turn it in to a data loader for accessing
    ypred = []
    ytruth = []
    xfeats = []
    with torch.no_grad():
        for feat, targ in dl:
            ypred.append(model(feat.cuda())[0].cpu().numpy())
            ytruth.append(targ.numpy())
            xfeats.append(feat.numpy())

    ypred = np.vstack(ypred)
    ytruth = np.vstack(ytruth)
    xfeats = np.vstack(xfeats)
    
    np.save(f"{folder_name}/{i}_xfeats.npy", xfeats)
    np.save(f"{folder_name}/{i}_ypred.npy", ypred)
    np.save(f"{folder_name}/{i}_ytruth.npy", ytruth)

# calculate feature importance first!!!

In [37]:
truth3d =  ytruth.reshape(39,159,169,-1)
pred3d =  ypred.reshape(39,159,169,-1)
pctdiff_cube = np.abs(np.exp((truth3d - pred3d)* tstd) - 1)

In [41]:
tstd

array([1.3428156, 1.0779657, 1.0479535])

In [42]:
from matplotlib.colors import ListedColormap, LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
from matplotlib import cm

for i in range(12):
    ytruth = np.load(f"{folder_name}/{i}_ytruth.npy")
    ypred  = np.load(f"{folder_name}/{i}_ypred.npy")
    xfeats = np.load(f"{folder_name}/{i}_xfeats.npy")
    truth3d =  ytruth.reshape(39,159,169,-1)
    pred3d =  ypred.reshape(39,159,169,-1)
    pctdiff_cube = np.abs(np.exp((truth3d - pred3d)* tstd) - 1)
    
    
    clf = TabNetRegressor(
    input_dim=len(feat_idx), 
    output_dim=len(target_idx), 
    n_d=8, 
    n_a=8
    )

    clf._set_network()
    clf._update_network_params()
    clf.network = model
    
    idx_important = []
    idx_value     = []

    for f in np.array_split(xfeats,100):
        t1 = time.time()
        explain_matrix, masks = clf.explain(f)
        norm_mat = explain_matrix/explain_matrix.sum(-1,keepdims=True)
        idx_important.append(norm_mat.argmax(axis=1))
        idx_value.append(norm_mat[np.arange(len(norm_mat)),norm_mat.argmax(axis=1)])
    #     print(time.time()-t1)

    imp = np.hstack(idx_important)

    uqix,uq_counts = np.unique(imp,return_counts=True)

    # select 5 most important predictor
    n_most_important = min(8, len(uq_counts))
    tmp = np.argsort(uq_counts)[::-1][:n_most_important]

    top_imp_idx    = uqix[tmp]
    top_imp_counts = uq_counts[tmp]

    ordered_imp = np.zeros_like(imp)+n_most_important
    for _i, _idx in enumerate(top_imp_idx):
        ordered_imp[imp == _idx] = _i
        
    cube_important = ordered_imp.reshape(39,159,169,-1)
    
    print(f"Starting {i} cube")
    for IDX_targ in range(len(target_idx)):
        print(f"Starting {IDX_targ} target")
        for zslice in range(39):
            f,ax = plt.subplots(1,4, figsize=(15,5))

            td = truth3d[zslice,:,:,0]

            im1 = ax.flat[0].imshow(truth3d[zslice, :,:,IDX_targ], vmin=td.min(), vmax=td.max())
            im2 = ax.flat[1].imshow(pred3d[zslice, :,:,IDX_targ], vmin=td.min(), vmax=td.max())

            divider = make_axes_locatable(ax.flat[0])
            cax = divider.append_axes("right", size="5%", pad=0.05)
            cbar = f.colorbar(im1, cax=cax, orientation='vertical')

            divider = make_axes_locatable(ax.flat[1])
            cax = divider.append_axes("right", size="5%", pad=0.05)
            cbar = f.colorbar(im1, cax=cax, orientation='vertical')

            im3 = ax.flat[2].imshow(pctdiff_cube[zslice, :,:,IDX_targ], vmin=1e-2,vmax=1e0,cmap='magma', norm= LogNorm(1e-2,1))
            divider = make_axes_locatable(ax.flat[2])
            cax = divider.append_axes("right", size="5%", pad=0.05)
            cbar = f.colorbar(im3, cax=cax, orientation='vertical')


            lcmap = cm.get_cmap('Pastel1')
            cMap = ListedColormap([lcmap(i) for i in range(n_most_important+1)])

            heatmap = ax.flat[3].imshow(cube_important[zslice, :,:,0]+0.5, cmap=cMap, vmin=0, vmax=n_most_important+1)

            divider = make_axes_locatable(ax.flat[3])
            cax = divider.append_axes("right", size="5%", pad=0.05)

            cbar = f.colorbar(heatmap, cax=cax, orientation='vertical')
            cbar.set_ticks(np.arange(n_most_important+1)+0.5)
            cbar.ax.set_yticklabels(list(featcols[feat_idx][top_imp_idx]) + ["Rest"])
            # cbar.set_label('# of contacts', rotation=270)

            ax.flat[2].set_title("percentage error")
            ax.flat[3].set_title(f"zslice = {zslice}; { targcols[target_idx][IDX_targ] }")

            ax.flat[0].set_title('truth')
            ax.flat[1].set_title('pred')
            ax.flat[0].set_xticks([])
            ax.flat[0].set_yticks([])

            ax.flat[1].set_xticks([])
            ax.flat[1].set_yticks([])
            ax.flat[2].set_xticks([])
            ax.flat[2].set_yticks([])

            ax.flat[3].set_xticks([])
            ax.flat[3].set_yticks([])
            # plt.colorbar()
            plt.savefig(f"{folder_name}/t{i}_z{zslice}_{targcols[target_idx][IDX_targ]}")
            plt.close(f)
# plt.close('all')

Starting 0 cube
Starting 0 target


/tmp/ipykernel_2879950/3836714511.py:73: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  im3 = ax.flat[2].imshow(pctdiff_cube[zslice, :,:,IDX_targ], vmin=1e-2,vmax=1e0,cmap='magma', norm= LogNorm(1e-2,1))


Starting 1 target
Starting 2 target
Starting 1 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 2 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 3 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 4 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 5 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 6 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 7 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 8 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 9 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 10 cube
Starting 0 target
Starting 1 target
Starting 2 target
Starting 11 cube
Starting 0 target
Starting 1 target
Starting 2 target


In [46]:
targcols

array(['ccn_001', 'ccn_003', 'ccn_006', 'CHI', 'CHI_CCN', 'D_ALPHA',
       'D_GAMMA', 'D_ALPHA_CCN', 'D_GAMMA_CCN', 'PM25'], dtype='<U11')